In [128]:
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import *
from keras.layers import *
from keras.layers.core import Dense
from keras.utils import plot_model


In [129]:
data = pd.read_csv("data_1000_3.csv") 
df = pd.DataFrame(data)

In [130]:
(trainX, testX) = train_test_split(df, test_size=0.2, random_state=42)
trainY = trainX['picked_node']
testY = testX['picked_node']


# Iteration 1: one hot encoded

In [131]:
def process_structured_data(df, train, test):

    continuous = ['left_node', 'right_node']
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    colourBin = LabelBinarizer().fit(df['left_colour'])
    trainLcolourX = colourBin.transform(train['left_colour'])
    testLcolourX = colourBin.transform(test['left_colour'])
    trainRcolourX = colourBin.transform(train['right_colour'])
    testRcolourX = colourBin.transform(test['right_colour'])
    trainX = np.hstack([trainX, trainLcolourX, trainRcolourX])
    testX = np.hstack([testX, testLcolourX, testRcolourX])
    
    return (trainX, testX)

In [132]:
(proc_trainX, proc_testX) = process_structured_data(df, trainX, testX)

In [80]:
def create_mlp(dim, regularizer=None):
    """Creates a simple two-layer MLP with inputs of the given dimension"""
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    model.add(Dense(1, activation="sigmoid", kernel_regularizer=regularizer))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [81]:
one_model = create_mlp(proc_trainX.shape[1])

In [82]:
one_model.fit(proc_trainX, trainY, batch_size=16, epochs=10, validation_data=(proc_testX, testY))

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 0s 130us/step - loss: 0.7018 - accuracy: 0.5763 - val_loss: 0.6635 - val_accuracy: 0.6600
Epoch 2/10
800/800 [==============================] - 0s 41us/step - loss: 0.6611 - accuracy: 0.6425 - val_loss: 0.6140 - val_accuracy: 0.7350
Epoch 3/10
800/800 [==============================] - 0s 36us/step - loss: 0.6223 - accuracy: 0.6812 - val_loss: 0.5684 - val_accuracy: 0.7600
Epoch 4/10
800/800 [==============================] - 0s 36us/step - loss: 0.5848 - accuracy: 0.7125 - val_loss: 0.5244 - val_accuracy: 0.7950
Epoch 5/10
800/800 [==============================] - 0s 39us/step - loss: 0.5478 - accuracy: 0.7362 - val_loss: 0.4836 - val_accuracy: 0.8100
Epoch 6/10
800/800 [==============================] - 0s 39us/step - loss: 0.5112 - accuracy: 0.7513 - val_loss: 0.4468 - val_accuracy: 0.8650
Epoch 7/10
800/800 [==============================] - 0s 39us/step - loss: 0.4792 - accuracy: 0

# Iteration 2: ordinal encoded

In [86]:
def process_integer_data(df, train, test):
    continuous = ['left_node', 'right_node']
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    colourencoder = OrdinalEncoder().fit(np.array(df['left_colour']).reshape(-1,1))
    trainLcolourX = colourencoder.transform(np.array(train['left_colour']).reshape(-1,1))
    testLcolourX = colourencoder.transform(np.array(test['left_colour']).reshape(-1,1))
    trainRcolourX = colourencoder.transform(np.array(train['right_colour']).reshape(-1,1))
    testRcolourX = colourencoder.transform(np.array(test['right_colour']).reshape(-1,1))
    trainX = np.hstack([trainX, trainLcolourX, trainRcolourX])
    testX = np.hstack([testX, testLcolourX, testRcolourX])
    
    return (trainX, testX)

In [90]:
(proc_trainX, proc_testX) = process_integer_data(df, trainX, testX)

In [88]:
int_model = create_mlp(proc_trainX.shape[1])

In [89]:
int_model.fit(proc_trainX, trainY, batch_size=10, epochs=10, validation_data=(proc_testX, testY))

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 0s 155us/step - loss: 0.6814 - accuracy: 0.5387 - val_loss: 0.6292 - val_accuracy: 0.7450
Epoch 2/10
800/800 [==============================] - 0s 60us/step - loss: 0.6331 - accuracy: 0.6675 - val_loss: 0.5789 - val_accuracy: 0.7450
Epoch 3/10
800/800 [==============================] - 0s 56us/step - loss: 0.5961 - accuracy: 0.6750 - val_loss: 0.5390 - val_accuracy: 0.7450
Epoch 4/10
800/800 [==============================] - 0s 56us/step - loss: 0.5657 - accuracy: 0.6812 - val_loss: 0.5087 - val_accuracy: 0.7650
Epoch 5/10
800/800 [==============================] - 0s 59us/step - loss: 0.5394 - accuracy: 0.7025 - val_loss: 0.4805 - val_accuracy: 0.8050
Epoch 6/10
800/800 [==============================] - 0s 60us/step - loss: 0.5126 - accuracy: 0.7650 - val_loss: 0.4490 - val_accuracy: 0.8500
Epoch 7/10
800/800 [==============================] - 0s 61us/step - loss: 0.4842 - accuracy: 0

# Iteration 3: Embedded

In [92]:
(proc_trainX, proc_testX) = process_integer_data(df, trainX, testX)

In [109]:
def emb(n):

    embedding_size = int(np.ceil(n/2))
    vocab  = n + 1

    model1 = Sequential()
    model1.add(Embedding(vocab ,embedding_size, input_length=1))
    model1.add(Reshape(target_shape=(embedding_size,)))

    model2 = Sequential()
    model2.add(Embedding(vocab ,embedding_size, input_length=1))
    model2.add(Reshape(target_shape=(embedding_size,)))

    model_rest = Sequential()
    model_rest.add(Dense(2, input_shape=(2,)))

    combined = concatenate([model_rest.output, model1.output, model2.output])

    x = Dense(8, activation="relu", kernel_initializer='he_normal')(combined)
    x = Dense(1, activation="sigmoid")(x)

    full_model = Model(inputs=[model_rest.input, model1.input, model2.input], outputs = x)
    full_model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer='adam')
    
    return full_model

emb_model = emb(len(np.unique(proc_trainX[:, 2])))

continous = proc_trainX[:, :2]
cat1 = proc_trainX[:, 2]
cat2 = proc_trainX[:, 3]

tcontinous = proc_testX[:, :2]
tcat1 = proc_testX[:, 2]
tcat2 = proc_testX[:, 3]

emb_model.fit([continous, cat1, cat2], trainY, epochs=5, batch_size=16, validation_data=([tcontinous, tcat1, tcat2], testY))

c:\python\python37\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 200 samples
Epoch 1/5
800/800 [==============================] - 0s 201us/step - loss: 0.7171 - acc: 0.4137 - val_loss: 0.6829 - val_acc: 0.6300
Epoch 2/5
800/800 [==============================] - 0s 55us/step - loss: 0.6609 - acc: 0.6675 - val_loss: 0.6191 - val_acc: 0.7450
Epoch 3/5
800/800 [==============================] - 0s 49us/step - loss: 0.6228 - acc: 0.6687 - val_loss: 0.5733 - val_acc: 0.7450
Epoch 4/5
800/800 [==============================] - 0s 49us/step - loss: 0.5934 - acc: 0.6687 - val_loss: 0.5321 - val_acc: 0.7450
Epoch 5/5
800/800 [==============================] - 0s 49us/step - loss: 0.5681 - acc: 0.6737 - val_loss: 0.5032 - val_acc: 0.7800


# Evaluation

In [133]:
def evaluate(integer=True, embed=False, ep=10):
    for i in range(2, 9):
        
        data = pd.read_csv("data_1000_" + str(i) + ".csv") 
        df = pd.DataFrame(data)
        
        (trainX, valX) = train_test_split(df, test_size=0.2, random_state=42)
        trainY = trainX['picked_node']
        valY = valX['picked_node']
        
        if integer:
            (proc_trainX, proc_valX) = process_integer_data(df, trainX, valX)
        else:
            (proc_trainX, proc_valX) = process_structured_data(df, trainX, valX)
        
        if embed:
            model = emb(len(np.unique(proc_trainX[:, 2])))
            continous = proc_trainX[:, :2]
            cat1 = proc_trainX[:, 2]
            cat2 = proc_trainX[:, 3]

            vcontinous = proc_valX[:, :2]
            vcat1 = proc_valX[:, 2]
            vcat2 = proc_valX[:, 3]

            model.fit([continous, cat1, cat2], epochs=ep, batch_size=16, validation_data=([vcontinous, vcat1, vcat2], valY))
        else:
            model = create_mlp(proc_trainX.shape[1])
            model.fit(proc_trainX, trainY, batch_size=10, epochs=ep, validation_data=(proc_valX, valY))
            
        data = pd.read_csv("test_200_" + str(i) + ".csv") 
        df= pd.DataFrame(data)
        
        (trainX, testX) = train_test_split(df, test_size=0.01, random_state=42)
        trainY = trainX['picked_node']
        testY = testX['picked_node']
        
        if integer:
            (proc_testX, proc_test2X) = process_integer_data(df, trainX, valX)
        else:
            (proc_testX, proc_test2X) = process_structured_data(df, trainX, valX)
        
        
        _, accuracy = model.evaluate(proc_testX, trainY, batch_size=128)
        
        print('#####################################################\n\n')
        print(str(i) + " colours accuracy = " + str(accuracy))
        print('\n\n#####################################################')

evaluate(ep=20)
        
        
        

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 0s 158us/step - loss: 0.6437 - accuracy: 0.6762 - val_loss: 0.6066 - val_accuracy: 0.6750
Epoch 2/20
800/800 [==============================] - 0s 65us/step - loss: 0.5909 - accuracy: 0.6575 - val_loss: 0.5542 - val_accuracy: 0.6900
Epoch 3/20
800/800 [==============================] - 0s 62us/step - loss: 0.5494 - accuracy: 0.7038 - val_loss: 0.5184 - val_accuracy: 0.7700
Epoch 4/20
800/800 [==============================] - 0s 59us/step - loss: 0.5106 - accuracy: 0.8188 - val_loss: 0.4788 - val_accuracy: 0.8450
Epoch 5/20
800/800 [==============================] - 0s 60us/step - loss: 0.4769 - accuracy: 0.8400 - val_loss: 0.4490 - val_accuracy: 0.8450
Epoch 6/20
800/800 [==============================] - 0s 62us/step - loss: 0.4498 - accuracy: 0.8400 - val_loss: 0.4239 - val_accuracy: 0.8450
Epoch 7/20
800/800 [==============================] - 0s 61us/step - loss: 0.4264 - accuracy: 0

800/800 [==============================] - 0s 60us/step - loss: 0.5260 - accuracy: 0.7475 - val_loss: 0.4980 - val_accuracy: 0.7750
Epoch 15/20
800/800 [==============================] - 0s 60us/step - loss: 0.5144 - accuracy: 0.7575 - val_loss: 0.4860 - val_accuracy: 0.7900
Epoch 16/20
800/800 [==============================] - 0s 59us/step - loss: 0.5038 - accuracy: 0.7775 - val_loss: 0.4730 - val_accuracy: 0.7950
Epoch 17/20
800/800 [==============================] - 0s 60us/step - loss: 0.4908 - accuracy: 0.7850 - val_loss: 0.4606 - val_accuracy: 0.8000
Epoch 18/20
800/800 [==============================] - 0s 60us/step - loss: 0.4800 - accuracy: 0.8100 - val_loss: 0.4487 - val_accuracy: 0.8050
Epoch 19/20
800/800 [==============================] - 0s 57us/step - loss: 0.4684 - accuracy: 0.8150 - val_loss: 0.4369 - val_accuracy: 0.8100
Epoch 20/20
198/198 [==============================] - 0s 5us/step
#####################################################


4 colours accuracy = 0.83

Epoch 6/20
800/800 [==============================] - 0s 60us/step - loss: 0.5424 - accuracy: 0.7337 - val_loss: 0.5829 - val_accuracy: 0.6900
Epoch 7/20
800/800 [==============================] - 0s 63us/step - loss: 0.5372 - accuracy: 0.7350 - val_loss: 0.5769 - val_accuracy: 0.7050
Epoch 8/20
800/800 [==============================] - 0s 62us/step - loss: 0.5331 - accuracy: 0.7412 - val_loss: 0.5733 - val_accuracy: 0.7100
Epoch 9/20
800/800 [==============================] - 0s 60us/step - loss: 0.5293 - accuracy: 0.7400 - val_loss: 0.5717 - val_accuracy: 0.7050
Epoch 10/20
800/800 [==============================] - 0s 64us/step - loss: 0.5257 - accuracy: 0.7437 - val_loss: 0.5692 - val_accuracy: 0.7000
Epoch 11/20
800/800 [==============================] - 0s 59us/step - loss: 0.5224 - accuracy: 0.7500 - val_loss: 0.5644 - val_accuracy: 0.7150
Epoch 12/20
800/800 [==============================] - 0s 62us/step - loss: 0.5191 - accuracy: 0.7487 - val_loss: 0.5608 - val_accuracy: 0.7